In [140]:
import pandas as pd      # para trabajar con archivos de datos csv, excel
import numpy as np

from sklearn import preprocessing, model_selection
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt

nombre_archivo = '../Datos/Vinos.csv'

df = pd.read_csv(nombre_archivo, sep=";")
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [141]:
X = np.array(df.iloc[:,1:])
T= np.array(df.iloc[:,:1])
print(X)

[[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]]


In [142]:
binarizer = preprocessing.LabelBinarizer() #para el onehot enconding
T = binarizer.fit_transform(T)
# print(T)

X_train, X_test, T_train, T_test = model_selection.train_test_split(X, T, test_size=0.2)#, random_state=42)
# print('\nDatos de Entrenamiento: %d   Datos de Testeo: %d' % (len(T_train), len(T_test) ))

scaler = preprocessing.StandardScaler()
#scaler = preprocessing.MinMaxScaler()

X_train  = scaler.fit_transform( X_train )
X_test   = scaler.transform( X_test )

print(X_train)

[[-0.25956398  0.54584903  0.90257213 ... -0.19629173 -0.53541354
  -0.54451593]
 [-0.87840356  0.10327865  0.32263891 ... -0.6005148   1.18646694
  -1.01530714]
 [ 0.30976843 -0.53020446 -0.91455197 ...  0.38758603  0.33252622
   1.60811493]
 ...
 [ 0.70582577  2.27274133 -0.10264546 ... -0.10646439 -0.67540382
  -0.66840835]
 [-1.20020015 -0.86864064 -0.29595653 ...  0.6121544   0.94848346
  -0.43611006]
 [ 0.27263806  1.69132533 -0.45060539 ... -1.54370196 -1.36135621
   0.21432517]]


In [143]:
#elijo parámetros de entrenamiento simialres a los que mostró el profe, siempre usando cosas que vimos por el momento. Aguante Relu loco

modelo = MLPClassifier(
    hidden_layer_sizes=(2),
    tol=1.0e-4,
    activation='relu',          # aguante relu, sklearn no tiene leaky relu
    solver='adam',              # todavía no vimos esto, pero el profe lo puso
    alpha=0.005,
    max_iter=2500,                        
).fit(X_train, T_train)

# Precision del entrenamiento

In [144]:
Y_pred = modelo.predict(X_train)
score = modelo.score(X_train, T_train)

# # "invierte" la transformacion binaria para obtener los nombres de las clases
Y_it = binarizer.inverse_transform(T_train)
Y_pred_it = binarizer.inverse_transform(Y_pred)

# Y_pred_it = np.argmax(Y_pred, axis = 1)

# calculo manual del accuracy
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
print('      Score: %6.2f' % (score) )
print(Y_pred_it)

Efectividad:  97.18%
      Score:   0.97
[3 2 1 1 2 2 2 1 2 1 3 2 3 3 1 3 1 2 3 2 2 3 1 2 3 3 1 1 2 3 2 2 2 1 1 1 2
 1 3 2 2 2 3 2 3 1 3 2 3 1 2 2 1 3 2 3 3 3 2 2 3 3 1 1 2 1 1 2 1 3 2 3 2 1
 3 1 1 1 3 1 1 3 1 3 1 1 2 1 2 1 1 2 3 1 3 3 3 3 3 3 1 2 1 2 1 1 2 2 2 2 1
 1 2 1 2 3 3 1 3 2 1 1 2 1 1 3 2 1 1 3 2 2 3 3 2 1 3 2 3 3 2 3]


# Precisión del testeo

In [145]:
Y_pred = modelo.predict(X_test)
score = modelo.score(X_test, T_test)

# "invierte" la transformacion binaria para obtener los nombres de las clases
Y_it = binarizer.inverse_transform(T_test)
Y_pred_it = binarizer.inverse_transform(Y_pred)

# calculo manual del accuracy
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
print('      Score: %6.2f' % (score) )

Efectividad: 100.00%
      Score:   1.00


# Resultados (en el orden que fui probando)

|capas ocultas|score train|score test|
|---|---|---|
|8, 4|0.99|0.94|
|8|0.99|0.94|
|4|0.97|1|
|2|0.99|0.92|
|3|0.99|0.94|
|1|0.68|0.72|

# Conclusión: los resultados los anoté al hacer pocas pruebas, mientras fui reiterando los entrenamientos noté que al usar una confuguración (2), (3) o (4) para las capas ocultas habían resultados similares, por lo cual al final me quedo con la arquitectura de (2)